# Curve Alignment ved ICP

In [28]:
import glob
import numpy as np
from scipy.spatial import KDTree
from scipy.linalg import svd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

# load kurver
paths = glob.glob("../data/3d kurver/*.csv")
kurver = [np.loadtxt(path, delimiter=",") for path in paths]


In [29]:
%matplotlib qt

## Plot Kurver Før Alignment

In [30]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

# Brug colormap med nok farver
n = len(kurver)
colors = cm.get_cmap('tab20', n)

# Plot de rå kurver
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

for i, curve in enumerate(kurver):
    ax.plot(*curve.T, color=colors(i), label=f"Curve {i+1}", linewidth=1.5)

ax.set_title("Curves before ICP alignment", fontsize=14)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0), fontsize='small')
plt.tight_layout()
plt.show()


C:\Users\lucca\AppData\Local\Temp\ipykernel_16684\3133343131.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = cm.get_cmap('tab20', n)


# Funktioner

In [31]:
def pca_align(points):
    centered = points - points.mean(axis=0)
    U, S, Vt = np.linalg.svd(centered)
    aligned = Vt @ centered.T
    return aligned.T, Vt

def icp_numpy(A_, B, max_iter=20, display=False, pca=False):
    if pca:
        A, Va = pca_align(A_)
        B_new, Vb = pca_align(B)
    else:
        A = A_
        B_new = B.copy()
    
    if display:
        fig = plt.figure(figsize=(5, 5))
        ax = fig.add_subplot(111, projection='3d')
        ax.view_init(elev=30, azim=100)
        ax.plot(*A.T, color='black', linewidth=3, label='Reference')
        ax.set_xlabel("X")
        ax.set_ylabel("Y")
        ax.set_zlabel("Z")
        plt.tight_layout()
    
    for i in range(max_iter):
        tree = KDTree(A)
        _, indices = tree.query(B_new)
        A_corr = A[indices]

        centroid_A = A_corr.mean(axis=0)
        centroid_B = B_new.mean(axis=0)

        AA = A_corr - centroid_A
        BB = B_new - centroid_B

        H = BB.T @ AA
        U, _, Vt = svd(H)
        R = Vt.T @ U.T

        if np.linalg.det(R) < 0:
            Vt[-1, :] *= -1
            R = Vt.T @ U.T

        t = centroid_A - R @ centroid_B
        B_new = (R @ B_new.T).T + t

        if display and i%5==0:
            ax.plot(*B_new.T, linewidth=2, label=f'Iteration {i+1}')
            ax.set_title(f"Iteration {i+1} vs. Reference", fontsize=14)
            ax.legend()
    return B_new

def resample_curve(curve, n_points=1000):
    from scipy.interpolate import interp1d
    d = np.cumsum(np.linalg.norm(np.diff(curve, axis=0), axis=1))
    d = np.insert(d, 0, 0)
    f = interp1d(d, curve, axis=0)
    d_new = np.linspace(0, d[-1], n_points)
    return f(d_new)

# Tjek hvilke kurver der er flippede

In [32]:
from sklearn.decomposition import PCA
import numpy as np

# 0-indekserede kurver du har markeret som spejlede
flip_pc1 = [2, 3, 5, 6, 9, 10, 11, 14, 18, 26, 27]
flip_pc2 = [3, 8, 12, 13, 14, 17, 18, 19, 20, 21, 22, 24, 26]
flip_pc3 = [9, 26]

# Step 1: PCA-rotation
def pca_align(curve):
    pca = PCA(n_components=3)
    centered = curve - np.mean(curve, axis=0)
    transformed = pca.fit_transform(centered)
    return transformed

# Step 2: Flip efter PCA-rotation
def flip_curve(curve, index):
    flipped = curve.copy()
    if index in flip_pc1:
        flipped[:, 0] *= -1
    if index in flip_pc2:
        flipped[:, 1] *= -1
    if index in flip_pc3:
        flipped[:, 2] *= -1
    return flipped

# Step 3: Resample + kør pipeline
resampled_curves = [resample_curve(c, 1000) for c in kurver]

# PCA + Flip
oriented_curves = []
for i, curve in enumerate(resampled_curves):
    pca_rotated = pca_align(curve)
    flipped = flip_curve(pca_rotated, i)
    oriented_curves.append(flipped)
reference = oriented_curves[0]
aligned_curves = [reference]

for i in range(1, len(oriented_curves)):
    aligned = icp_numpy(reference, oriented_curves[i], max_iter=50)
    aligned_curves.append(aligned)



# PLot alle kurver i et

In [33]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

for i, curve in enumerate(aligned_curves):
    ax.plot(*curve.T, linewidth=1.2, label=f'Kurve {i+1}')

ax.set_title("After PCA → Flip → ICP", fontsize=14)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
#ax.view_init(elev=30, azim=100)
plt.tight_layout()
plt.show()

### Med PC elementer

In [34]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

# Brug den PCA-roterede, men IKKE ICP-alignede referencekurve
reference_pca = oriented_curves[0]  # efter PCA + flip, før ICP

# Beregn PCA på den – for at få komponenter i 'rå PCA-orientering'
pca = PCA(n_components=3)
pca.fit(reference_pca)

components = pca.components_        # PC1, PC2, PC3
center = reference_pca.mean(axis=0)  # centerpunkt for at placere pile
explained = pca.explained_variance_ratio_

# Skaleringsfaktor til pile
scale = 50  # justér alt efter plot

# ---------- Plot aligned kurver + PCA-aksen ----------
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot alle kurver (alignede)
for i, curve in enumerate(aligned_curves):
    ax.plot(*curve.T, linewidth=1.2, label=f'Curve {i+1}')

# Tegn PCA-aksen fra præ-ICP kurven
colors = ['red', 'green', 'blue']
labels = ['PC1', 'PC2', 'PC3']
for i in range(3):
    vec = components[i] * scale
    ax.quiver(*center, *vec, color=colors[i], linewidth=2.5, arrow_length_ratio=0.08, label=labels[i])

# Layout og stil
ax.set_title("Aligned Curves + PCA axes (from pre-ICP reference)", fontsize=14)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_ylim([-50, 50])
ax.set_zlabel("Z")
ax.view_init(elev=0, azim=180)
#ax.legend(loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()

# 📊 Vis evt. hvor meget variance hver PC forklarer
print("Explained variance ratio:")
for label, var in zip(labels, explained):
    print(f"{label}: {var*100:.2f} %")


Explained variance ratio:
PC1: 87.62 %
PC2: 9.91 %
PC3: 2.48 %


# Roter om PC1


In [35]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from scipy.spatial.transform import Rotation as R

# ---------- Roteringsfunktion ----------
def rotate_around_axis(points, axis, angle_deg, point_on_axis=None):
    axis = axis / np.linalg.norm(axis)
    if point_on_axis is None:
        point_on_axis = np.zeros(3)
    centered = points - point_on_axis
    rot = R.from_rotvec(np.deg2rad(angle_deg) * axis)
    rotated = rot.apply(centered)
    return rotated + point_on_axis

# ---------- PCA på reference før ICP ----------
reference_pca = oriented_curves[0]  # før ICP
pca = PCA(n_components=3)
pca.fit(reference_pca)
components = pca.components_  # PC1, PC2, PC3
center = reference_pca.mean(axis=0)
pc1 = components[0]

# ---------- Roter alle alignede kurver ----------
angle_deg = -60  # justér efter behov
rotated_curves = [rotate_around_axis(curve, axis=pc1, angle_deg=angle_deg, point_on_axis=center)
                  for curve in aligned_curves]

# ---------- Plot de roterede kurver + PCA-aksene ----------
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot kurver
for i, curve in enumerate(rotated_curves):
    ax.plot(*curve.T, linewidth=1.2, label=f'Curve {i+1}')

# Plot PCA-aksene
colors = ['red', 'green', 'blue']
labels = ['PC1', 'PC2', 'PC3']
scale = 50  # justér længden på vektorerne

for i in range(3):
    vec = components[i] * scale
    ax.quiver(*center, *vec, color=colors[i], linewidth=3, arrow_length_ratio=0.08, label=labels[i])

# Styling
ax.set_title(f"Curves Rotated {angle_deg}° around PC1", fontsize=14)
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_ylim([-50, 50])
ax.set_zlabel("Z")
ax.view_init(elev=0, azim=180)
#ax.legend(loc='upper left', fontsize='small')
plt.tight_layout()
plt.show()


# Gem


In [36]:
import os
import numpy as np

# Eksempel: paths = glob.glob("3d kurver/*.csv")
# Du har brugt dem til at loade dine kurver
# Nu bruger vi samme navne til at gemme aligned versioner

output_folder = "Aligned 3d kurver"
os.makedirs(output_folder, exist_ok=True)

for i, path in enumerate(paths):
    filename = os.path.basename(path)             # F.eks. "curve1.csv"
    name, ext = os.path.splitext(filename)        # Split: "curve1", ".csv"
    new_name = f"{name}_aligned{ext}"             # F.eks. "curve1_aligned.csv"
    save_path = os.path.join(output_folder, new_name)

    np.savetxt(save_path, aligned_curves[i], delimiter=",")
    print(f"✅ Gemte {new_name}")

✅ Gemte Rør10Kurve1_3d_aligned.csv
✅ Gemte Rør10Kurve2_3d_aligned.csv
✅ Gemte Rør10Kurve3_3d_aligned.csv
✅ Gemte Rør10Kurve4_3d_aligned.csv
✅ Gemte Rør1Kurve4_3d_aligned.csv
✅ Gemte Rør1Kurve5_3d_aligned.csv
✅ Gemte Rør2Kurve3_3d_aligned.csv
✅ Gemte Rør2Kurve4_3d_aligned.csv
✅ Gemte Rør3Kurve1_3d_aligned.csv
✅ Gemte Rør3Kurve4_3d_aligned.csv
✅ Gemte Rør4Kurve1_3d_aligned.csv
✅ Gemte Rør4Kurve2_3d_aligned.csv
✅ Gemte Rør5Kurve3_3d_aligned.csv
✅ Gemte Rør5Kurve4_3d_aligned.csv
✅ Gemte Rør5Kurve5_3d_aligned.csv
✅ Gemte Rør6Kurve2_3d_aligned.csv
✅ Gemte Rør6Kurve4_3d_aligned.csv
✅ Gemte Rør6Kurve5_3d_aligned.csv
✅ Gemte Rør7Kurve1_3d_aligned.csv
✅ Gemte Rør7Kurve2_3d_aligned.csv
✅ Gemte Rør7Kurve3_3d_aligned.csv
✅ Gemte Rør7Kurve4_3d_aligned.csv
✅ Gemte Rør8Kurve1_3d_aligned.csv
✅ Gemte Rør8Kurve3_3d_aligned.csv
✅ Gemte Rør8Kurve4_3d_aligned.csv
✅ Gemte Rør8Kurve5_3d_aligned.csv
✅ Gemte Rør9Kurve2_3d_aligned.csv
✅ Gemte Rør9Kurve4_3d_aligned.csv


In [37]:
import os
import re
import numpy as np

output_folder = "PC roteret"
os.makedirs(output_folder, exist_ok=True)

for i, path in enumerate(paths):
    filename = os.path.basename(path)             # Fx: "Rør1Kurve2.csv"
    name, ext = os.path.splitext(filename)        # Fx: "Rør1Kurve2", ".csv"

    # Find Rør-nummer og Kurve-nummer
    match = re.match(r"Rør(\d+)(Kurve\d+)", name)
    if match:
        rør_nr = int(match.group(1))
        kurve_part = match.group(2)

        new_name = f"Rør{rør_nr:02d}{kurve_part}_PC{ext}"  # Fx: Rør01Kurve2_PC.csv
    else:
        new_name = f"{name}_PC{ext}"  # fallback hvis regex fejler

    save_path = os.path.join(output_folder, new_name)
    np.savetxt(save_path, rotated_curves[i], delimiter=",")
    print(f"✅ Gemte {new_name}")


✅ Gemte Rør10Kurve1_PC.csv
✅ Gemte Rør10Kurve2_PC.csv
✅ Gemte Rør10Kurve3_PC.csv
✅ Gemte Rør10Kurve4_PC.csv
✅ Gemte Rør01Kurve4_PC.csv
✅ Gemte Rør01Kurve5_PC.csv
✅ Gemte Rør02Kurve3_PC.csv
✅ Gemte Rør02Kurve4_PC.csv
✅ Gemte Rør03Kurve1_PC.csv
✅ Gemte Rør03Kurve4_PC.csv
✅ Gemte Rør04Kurve1_PC.csv
✅ Gemte Rør04Kurve2_PC.csv
✅ Gemte Rør05Kurve3_PC.csv
✅ Gemte Rør05Kurve4_PC.csv
✅ Gemte Rør05Kurve5_PC.csv
✅ Gemte Rør06Kurve2_PC.csv
✅ Gemte Rør06Kurve4_PC.csv
✅ Gemte Rør06Kurve5_PC.csv
✅ Gemte Rør07Kurve1_PC.csv
✅ Gemte Rør07Kurve2_PC.csv
✅ Gemte Rør07Kurve3_PC.csv
✅ Gemte Rør07Kurve4_PC.csv
✅ Gemte Rør08Kurve1_PC.csv
✅ Gemte Rør08Kurve3_PC.csv
✅ Gemte Rør08Kurve4_PC.csv
✅ Gemte Rør08Kurve5_PC.csv
✅ Gemte Rør09Kurve2_PC.csv
✅ Gemte Rør09Kurve4_PC.csv
